# Mini CNN Image Classifier + Topology (Persistent Homology) Detector (Synthetic Images)

This notebook tests the repo’s **topology-based detector** (`detector_type='topology_score'`) on an **image-like** setting using a **fully synthetic 2-class RGB dataset**.

In [ ]:
import os
import sys
import importlib
import numpy as np

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Project root
CWD = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(CWD, "..")) if os.path.basename(CWD) == "notebooks" else os.path.abspath(CWD)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Public pipeline API
from src import api
from src.utils import ExperimentConfig, set_seed

from src.models import extract_features_batch
from src.visualization import plot_roc_from_metrics, plot_confusion_matrix, plot_score_distributions_figure

In [ ]:
# Configuration
SEED = 42
set_seed(SEED)

DEVICE = "cpu"

# Dataset sizes
N_TRAIN = 2000
N_VAL = 500
N_TEST = 500

# Training
EPOCHS = 8
BATCH_SIZE = 64
LR = 1e-3

# Attack (inputs in [0,1])
ATTACK_TYPE = 'fgsm'  # 'fgsm' or 'pgd'
EPSILON = 0.03
PGD_STEPS = 10
PGD_STEP_SIZE = 0.007

# Topology detector settings
TOPO_K = 40
TOPO_MAXDIM = 1
TOPO_PREPROCESS = 'pca'
TOPO_PCA_DIM = 10
TOPO_MIN_PERSISTENCE = 1e-6

TOPO_PERCENTILE = 95.0
TOPO_COV_SHRINKAGE = 1e-3

# PH can be expensive; subsample scoring points
MAX_POINTS_FOR_SCORING = 250

print(f"Attack: {ATTACK_TYPE}, eps={EPSILON}")
print(f"Topology: k={TOPO_K}, maxdim={TOPO_MAXDIM}, preprocess={TOPO_PREPROCESS}({TOPO_PCA_DIM})")

In [ ]:
# Helpers
from typing import Tuple


def plot_samples(X: np.ndarray, y: np.ndarray, *, title: str, n_show: int = 8):
    fig, axes = plt.subplots(1, n_show, figsize=(2.2 * n_show, 2.2))
    for i in range(n_show):
        img = X[i].transpose(1, 2, 0)
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f"y={int(y[i])}")
        axes[i].axis("off")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def compute_success_mask(model: nn.Module, X_clean: np.ndarray, X_adv: np.ndarray, y: np.ndarray, device: str):
    pred_clean = api.predict(model, X_clean, device=device, return_probs=False)
    pred_adv = api.predict(model, X_adv, device=device, return_probs=False)
    y = np.asarray(y, dtype=int)
    clean_correct = pred_clean == y
    success = api.attack_success_mask(model, X_clean, X_adv, y, device=device)
    return pred_clean, pred_adv, clean_correct, success


def show_successful_attacks(
    X_clean: np.ndarray,
    X_adv: np.ndarray,
    y_true: np.ndarray,
    pred_clean: np.ndarray,
    pred_adv: np.ndarray,
    success_mask: np.ndarray,
    *,
    epsilon: float,
    n_show: int = 6,
    seed: int = 0,
):
    idx = np.where(success_mask)[0]
    if len(idx) == 0:
        print("No successful attacks to show.")
        return
    rng = np.random.default_rng(seed)
    chosen = rng.choice(idx, size=min(n_show, len(idx)), replace=False)

    fig, axes = plt.subplots(len(chosen), 2, figsize=(7, 3 * len(chosen)))
    if len(chosen) == 1:
        axes = np.expand_dims(axes, axis=0)

    for r, i in enumerate(chosen):
        clean = np.clip(X_clean[i].transpose(1, 2, 0), 0, 1)
        adv = np.clip(X_adv[i].transpose(1, 2, 0), 0, 1)
        axes[r, 0].imshow(clean)
        axes[r, 0].axis("off")
        axes[r, 0].set_title(f"clean y={int(y_true[i])}, pred={int(pred_clean[i])}")
        axes[r, 1].imshow(adv)
        axes[r, 1].axis("off")
        axes[r, 1].set_title(f"adv pred={int(pred_adv[i])} (eps={epsilon})")

    plt.tight_layout()
    plt.show()


In [ ]:
# Load dataset (via public dataset registry)
DATASET_NAME = "synthetic_shapes_2class"

_data_cfg = ExperimentConfig(seed=SEED, device=DEVICE)
_total = int(N_TRAIN + N_VAL + N_TEST)
_data_cfg.data.n_samples = _total
_data_cfg.data.train_ratio = float(N_TRAIN) / float(_total)
_data_cfg.data.val_ratio = float(N_VAL) / float(_total)

bundle = api.get_dataset(DATASET_NAME, _data_cfg)
X_train, y_train = bundle.X_train, bundle.y_train
X_val, y_val = bundle.X_val, bundle.y_val
X_test, y_test = bundle.X_test, bundle.y_test

print(f"Dataset: {DATASET_NAME}")
print(f"X_train: {X_train.shape} | y mean: {y_train.mean():.3f}")
print(f"X_val:   {X_val.shape} | y mean: {y_val.mean():.3f}")
print(f"X_test:  {X_test.shape} | y mean: {y_test.mean():.3f}")

In [ ]:
# Visual sanity-check
plot_samples(X_train, y_train, title=f"{DATASET_NAME}: first samples", n_show=8)

In [ ]:
# Model + experiment config (API)
config = ExperimentConfig(seed=SEED, device=DEVICE)

config.model.epochs = EPOCHS
config.model.batch_size = BATCH_SIZE
config.model.learning_rate = LR
config.model.output_dim = 2

config.attack.attack_type = ATTACK_TYPE
config.attack.epsilon = EPSILON
config.attack.num_steps = PGD_STEPS
config.attack.step_size = PGD_STEP_SIZE
config.attack.random_start = True

config.graph.space = "feature"
config.graph.use_topology = True
config.graph.use_tangent = False
config.graph.topo_k = TOPO_K
config.graph.topo_maxdim = TOPO_MAXDIM
config.graph.topo_preprocess = TOPO_PREPROCESS
config.graph.topo_pca_dim = TOPO_PCA_DIM
config.graph.topo_min_persistence = TOPO_MIN_PERSISTENCE

config.detector.detector_type = "topology_score"
config.detector.topo_percentile = TOPO_PERCENTILE
config.detector.topo_cov_shrinkage = TOPO_COV_SHRINKAGE

model = api.get_model("CNN", config, num_classes=2, in_channels=3, feat_dim=128)
print(model)

In [ ]:
# Train (API)
# Note: `bundle` was created in the dataset cell.
model, history = api.train(
    model,
    bundle,
    config,
    device=str(config.device),
    verbose=True,
    return_history=True,
)

pred_test = api.predict(model, X_test, device=str(config.device), return_probs=False)
test_acc = 100.0 * float(np.mean(pred_test == np.asarray(y_test, dtype=int)))
print(f"Test accuracy: {test_acc:.2f}%")

In [ ]:
# Adversarial examples + success masks (API)
X_adv_val = api.generate_adversarial(model, X_val, y_val, config, clip=bundle.meta.get("clip"))
X_adv_test = api.generate_adversarial(model, X_test, y_test, config, clip=bundle.meta.get("clip"))

EVAL_ONLY_SUCCESSFUL_ATTACKS = True
FILTER_CLEAN_TO_CORRECT = True

pred_val_clean, pred_val_adv, val_clean_correct, val_success = compute_success_mask(
    model, X_val, X_adv_val, y_val, device=str(config.device)
)
pred_test_clean, pred_test_adv, test_clean_correct, test_success = compute_success_mask(
    model, X_test, X_adv_test, y_test, device=str(config.device)
)

print(f"Val successful attacks:  {val_success.sum()}/{len(val_success)} = {val_success.mean():.3f}")
print(f"Test successful attacks: {test_success.sum()}/{len(test_success)} = {test_success.mean():.3f}")

In [ ]:
# Visualize a few successful attacks
show_successful_attacks(
    X_test, X_adv_test, y_test,
    pred_test_clean, pred_test_adv,
    test_success,
    epsilon=EPSILON,
    n_show=6,
    seed=SEED,
)

In [ ]:
# Reference embedding set from TRAIN
Z_train = extract_features_batch(model, X_train, layer='penultimate', device=DEVICE)
print(f"Z_train: {Z_train.shape}")

In [ ]:
# Topology-score detector (API)
clean_mask_val = val_clean_correct if FILTER_CLEAN_TO_CORRECT else np.ones(len(X_val), dtype=bool)
clean_mask_test = test_clean_correct if FILTER_CLEAN_TO_CORRECT else np.ones(len(X_test), dtype=bool)
adv_mask_val = val_success if EVAL_ONLY_SUCCESSFUL_ATTACKS else np.ones(len(X_adv_val), dtype=bool)
adv_mask_test = test_success if EVAL_ONLY_SUCCESSFUL_ATTACKS else np.ones(len(X_adv_test), dtype=bool)

X_val_clean_used, _ = api.subsample_masked(X_val, y_val, clean_mask_val, MAX_POINTS_FOR_SCORING, seed=SEED)
X_test_clean_used, _ = api.subsample_masked(X_test, y_test, clean_mask_test, MAX_POINTS_FOR_SCORING, seed=SEED + 1)
X_val_adv_used, _ = api.subsample_masked(X_adv_val, y_val, adv_mask_val, MAX_POINTS_FOR_SCORING, seed=SEED + 2)
X_test_adv_used, _ = api.subsample_masked(X_adv_test, y_test, adv_mask_test, MAX_POINTS_FOR_SCORING, seed=SEED + 3)

scores_val_clean = api.compute_scores(X_val_clean_used, model, bundle=bundle, cfg=config)
scores_val_adv = api.compute_scores(X_val_adv_used, model, bundle=bundle, cfg=config)

scores_test_clean = api.compute_scores(X_test_clean_used, model, bundle=bundle, cfg=config)
scores_test_adv = api.compute_scores(X_test_adv_used, model, bundle=bundle, cfg=config)

scores_val_all = api.concat_scores(scores_val_clean, scores_val_adv)
any_key = next(iter(scores_val_all.keys()))
labels_val = np.concatenate(
    [np.zeros(len(scores_val_clean[any_key]), dtype=int), np.ones(len(scores_val_adv[any_key]), dtype=int)]
)

detector = api.fit_detector(scores_val_all, labels_val, config)

scores_test_all = api.concat_scores(scores_test_clean, scores_test_adv)
any_key_t = next(iter(scores_test_all.keys()))
labels_test = np.concatenate(
    [np.zeros(len(scores_test_clean[any_key_t]), dtype=int), np.ones(len(scores_test_adv[any_key_t]), dtype=int)]
)

raw_scores_test = np.asarray(detector.score(scores_test_all), dtype=float)
metrics = api.evaluate_detection(labels_test, raw_scores_test)

print("Detector performance (topology_mahalanobis):")
print(f"  AUROC:     {metrics['roc_auc']:.4f}")
print(f"  AUPRC:     {metrics['pr_auc']:.4f}")
print(f"  FPR@95TPR: {metrics['fpr_at_tpr95']:.4f}")


# Mini CNN Image Classifier + Topology (Persistent Homology) Detector (Synthetic Images)

This notebook extends the repository’s **topology-based detector** (`detector_type='topology_score'`) to an **image-like** setting using a **fully synthetic 2-class RGB dataset**.

We will:

- generate a simple synthetic image dataset (circles vs squares),
- train a small CNN,
- generate adversarial examples (FGSM/PGD) in pixel space,
- extract penultimate-layer embeddings,
- compute local persistent-homology (PH) summary features on kNN neighborhoods in embedding space,
- fit a `TopologyScoreDetector` (Mahalanobis-to-clean + percentile threshold),
- evaluate detection metrics (AUROC/AUPRC/FPR@95%TPR).

Notes:
- This uses **no torchvision** and performs **no dataset downloads**.
- Persistent homology requires `ripser` (already in this repo’s `requirements.txt`).

In [ ]:
import os
import sys
import importlib
import numpy as np

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Project root
CWD = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(CWD, "..")) if os.path.basename(CWD) == "notebooks" else os.path.abspath(CWD)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Public pipeline API
from src import api
from src.utils import ExperimentConfig, set_seed

from src.models import extract_features_batch
from src.visualization import plot_roc_from_metrics, plot_confusion_matrix, plot_score_distributions_figure

In [ ]:
# Configuration
SEED = 42
set_seed(SEED)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataset sizes (keep small for fast CPU runs + PH runtime)
N_TRAIN = 2000
N_VAL = 500
N_TEST = 500

# Model training
EPOCHS = 8
BATCH_SIZE = 64
LR = 1e-3

# Attack settings (pixel-space; inputs in [0,1])
ATTACK_TYPE = 'fgsm'  # 'fgsm' or 'pgd'
EPSILON = 0.03        # typical for [0,1] images
PGD_STEPS = 10
PGD_STEP_SIZE = 0.007

# Topology detector settings (computed in feature space)
TOPO_K = 40                 # neighborhood size for local PH
TOPO_MAXDIM = 1             # H0/H1
TOPO_PREPROCESS = 'pca'      # helps in high-d embedding spaces
TOPO_PCA_DIM = 10
TOPO_MIN_PERSISTENCE = 1e-6

TOPO_PERCENTILE = 95.0       # clean-score percentile threshold
TOPO_COV_SHRINKAGE = 1e-3

# Scoring subsampling (PH per point can be expensive)
MAX_POINTS_FOR_SCORING = 250

print(f"DEVICE: {DEVICE}")
print(f"Train/Val/Test: {N_TRAIN}/{N_VAL}/{N_TEST}")
print(f"Attack: {ATTACK_TYPE}, eps={EPSILON}")
print(f"Topology: k={TOPO_K}, maxdim={TOPO_MAXDIM}, preprocess={TOPO_PREPROCESS}({TOPO_PCA_DIM})")

In [ ]:
# Helpers (keep all reusable functions/classes here) — API version
from typing import Tuple


def load_dataset(
    *,
    n_train: int,
    n_val: int,
    n_test: int,
    seed: int,
) -> Tuple[str, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    data_cfg = ExperimentConfig(seed=seed, device=DEVICE)
    total = int(n_train + n_val + n_test)
    data_cfg.data.n_samples = total
    data_cfg.data.train_ratio = float(n_train) / float(total)
    data_cfg.data.val_ratio = float(n_val) / float(total)

    b = api.get_dataset("synthetic_shapes_2class", data_cfg)
    return "synthetic_shapes_2class", b.X_train, b.y_train, b.X_val, b.y_val, b.X_test, b.y_test


def plot_samples(X: np.ndarray, y: np.ndarray, *, title: str, n_show: int = 8):
    fig, axes = plt.subplots(1, n_show, figsize=(2.2 * n_show, 2.2))
    for i in range(n_show):
        img = X[i].transpose(1, 2, 0)
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f"y={int(y[i])}")
        axes[i].axis("off")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def compute_success_mask(model: nn.Module, X_clean: np.ndarray, X_adv: np.ndarray, y: np.ndarray, device: str):
    pred_clean = api.predict(model, X_clean, device=device, return_probs=False)
    pred_adv = api.predict(model, X_adv, device=device, return_probs=False)
    y = np.asarray(y, dtype=int)
    clean_correct = pred_clean == y
    success = api.attack_success_mask(model, X_clean, X_adv, y, device=device)
    return pred_clean, pred_adv, clean_correct, success


def show_successful_attacks(
    X_clean: np.ndarray,
    X_adv: np.ndarray,
    y_true: np.ndarray,
    pred_clean: np.ndarray,
    pred_adv: np.ndarray,
    success_mask: np.ndarray,
    *,
    epsilon: float,
    n_show: int = 6,
    seed: int = 0,
):
    idx = np.where(success_mask)[0]
    if len(idx) == 0:
        print("No successful attacks to show. Try increasing EPSILON / PGD_STEPS.")
        return

    rng = np.random.default_rng(seed)
    chosen = rng.choice(idx, size=min(n_show, len(idx)), replace=False)

    fig, axes = plt.subplots(len(chosen), 2, figsize=(7, 3 * len(chosen)))
    if len(chosen) == 1:
        axes = np.expand_dims(axes, axis=0)

    for r, i in enumerate(chosen):
        clean = np.clip(X_clean[i].transpose(1, 2, 0), 0, 1)
        adv = np.clip(X_adv[i].transpose(1, 2, 0), 0, 1)

        axes[r, 0].imshow(clean)
        axes[r, 0].axis("off")
        axes[r, 0].set_title(f"clean: y={int(y_true[i])}, pred={int(pred_clean[i])}")

        axes[r, 1].imshow(adv)
        axes[r, 1].axis("off")
        axes[r, 1].set_title(f"adv: pred={int(pred_adv[i])}  (eps={epsilon})")

    plt.suptitle(f"Successful attacks (n={len(idx)})")
    plt.tight_layout()
    plt.show()


In [ ]:
# Load dataset (via public dataset registry)
DATASET_NAME = "synthetic_shapes_2class"

_data_cfg = ExperimentConfig(seed=SEED, device=DEVICE)
_total = int(N_TRAIN + N_VAL + N_TEST)
_data_cfg.data.n_samples = _total
_data_cfg.data.train_ratio = float(N_TRAIN) / float(_total)
_data_cfg.data.val_ratio = float(N_VAL) / float(_total)

bundle = api.get_dataset(DATASET_NAME, _data_cfg)
X_train, y_train = bundle.X_train, bundle.y_train
X_val, y_val = bundle.X_val, bundle.y_val
X_test, y_test = bundle.X_test, bundle.y_test

print(f"Dataset: {DATASET_NAME}")
print(f"X_train: {X_train.shape}, y_train mean: {y_train.mean():.3f}")
print(f"X_val:   {X_val.shape}, y_val mean:   {y_val.mean():.3f}")
print(f"X_test:  {X_test.shape}, y_test mean:  {y_test.mean():.3f}")


In [ ]:
# Visual sanity-check
plot_samples(X_train, y_train, title=f"{DATASET_NAME}: first samples", n_show=8)


In [ ]:
# Model + experiment config (API)
config = ExperimentConfig(seed=SEED, device=DEVICE)

# Training
config.model.epochs = EPOCHS
config.model.batch_size = BATCH_SIZE
config.model.learning_rate = LR
config.model.output_dim = 2

# Attack
config.attack.attack_type = ATTACK_TYPE
config.attack.epsilon = EPSILON
config.attack.num_steps = PGD_STEPS
config.attack.step_size = PGD_STEP_SIZE
config.attack.random_start = True

# Topology features computed in feature space
config.graph.space = "feature"
config.graph.use_topology = True
config.graph.use_tangent = False
config.graph.topo_k = TOPO_K
config.graph.topo_maxdim = TOPO_MAXDIM
config.graph.topo_preprocess = TOPO_PREPROCESS
config.graph.topo_pca_dim = TOPO_PCA_DIM
config.graph.topo_min_persistence = TOPO_MIN_PERSISTENCE

# Topology-score detector
config.detector.detector_type = "topology_score"
config.detector.topo_percentile = TOPO_PERCENTILE
config.detector.topo_cov_shrinkage = TOPO_COV_SHRINKAGE

model = api.get_model("CNN", config, num_classes=2, in_channels=3, feat_dim=128)
print(model)


In [ ]:
# Train (API)
model, history = api.train(
    model,
    bundle,
    config,
    device=str(config.device),
    verbose=True,
    return_history=True,
)

pred_test = api.predict(model, X_test, device=str(config.device), return_probs=False)
test_acc = 100.0 * float(np.mean(pred_test == np.asarray(y_test, dtype=int)))
print(f"Test accuracy: {test_acc:.2f}%")

# (Optional) training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history['train_loss'], label='train'); axes[0].plot(history['val_loss'], label='val'); axes[0].set_title('Loss'); axes[0].grid(True, alpha=0.3); axes[0].legend()
axes[1].plot(history['train_acc'], label='train'); axes[1].plot(history['val_acc'], label='val'); axes[1].set_title('Accuracy (%)'); axes[1].grid(True, alpha=0.3); axes[1].legend()
plt.tight_layout(); plt.show()


In [ ]:
# Adversarial examples + success masks (API)
X_adv_val = api.generate_adversarial(model, X_val, y_val, config, clip=bundle.meta.get("clip"))
X_adv_test = api.generate_adversarial(model, X_test, y_test, config, clip=bundle.meta.get("clip"))

# Successful = model correct on clean AND wrong on adv
EVAL_ONLY_SUCCESSFUL_ATTACKS = True
FILTER_CLEAN_TO_CORRECT = True

pred_val_clean, pred_val_adv, val_clean_correct, val_success = compute_success_mask(
    model, X_val, X_adv_val, y_val, device=str(config.device)
)
pred_test_clean, pred_test_adv, test_clean_correct, test_success = compute_success_mask(
    model, X_test, X_adv_test, y_test, device=str(config.device)
)

print(f"Val successful attacks:  {val_success.sum()}/{len(val_success)} = {val_success.mean():.3f}")
print(f"Test successful attacks: {test_success.sum()}/{len(test_success)} = {test_success.mean():.3f}")


In [ ]:
# Visualize a few successful attacks
show_successful_attacks(
    X_test,
    X_adv_test,
    y_test,
    pred_test_clean,
    pred_test_adv,
    test_success,
    epsilon=EPSILON,
    n_show=6,
    seed=SEED,
)


In [ ]:
# Reference embedding set from TRAIN
Z_train = extract_features_batch(model, X_train, layer='penultimate', device=DEVICE)
print(f"Z_train: {Z_train.shape}")


In [ ]:
# Topology-score detector (PH feature vector -> Mahalanobis -> threshold) — API

# Choose which points participate in scoring
clean_mask_val = val_clean_correct if FILTER_CLEAN_TO_CORRECT else np.ones(len(X_val), dtype=bool)
clean_mask_test = test_clean_correct if FILTER_CLEAN_TO_CORRECT else np.ones(len(X_test), dtype=bool)

adv_mask_val = val_success if EVAL_ONLY_SUCCESSFUL_ATTACKS else np.ones(len(X_adv_val), dtype=bool)
adv_mask_test = test_success if EVAL_ONLY_SUCCESSFUL_ATTACKS else np.ones(len(X_adv_test), dtype=bool)

seed0 = int(getattr(config, "seed", 42))
X_val_clean_used, _ = api.subsample_masked(X_val, y_val, clean_mask_val, MAX_POINTS_FOR_SCORING, seed=seed0)
X_test_clean_used, _ = api.subsample_masked(X_test, y_test, clean_mask_test, MAX_POINTS_FOR_SCORING, seed=seed0 + 1)
X_val_adv_used, _ = api.subsample_masked(X_adv_val, y_val, adv_mask_val, MAX_POINTS_FOR_SCORING, seed=seed0 + 2)
X_test_adv_used, _ = api.subsample_masked(X_adv_test, y_test, adv_mask_test, MAX_POINTS_FOR_SCORING, seed=seed0 + 3)

print(f"Scoring sizes: val clean={len(X_val_clean_used)}, val adv={len(X_val_adv_used)}")
print(f"Scoring sizes: test clean={len(X_test_clean_used)}, test adv={len(X_test_adv_used)}")

# Compute scores (includes topo_* keys when config.graph.use_topology=True)
scores_val_clean = api.compute_scores(X_val_clean_used, model, bundle=bundle, cfg=config)
scores_val_adv = api.compute_scores(X_val_adv_used, model, bundle=bundle, cfg=config)

scores_test_clean = api.compute_scores(X_test_clean_used, model, bundle=bundle, cfg=config)
scores_test_adv = api.compute_scores(X_test_adv_used, model, bundle=bundle, cfg=config)

# Train detector on val
scores_val_all = api.concat_scores(scores_val_clean, scores_val_adv)
any_key = next(iter(scores_val_all.keys()))
labels_val = np.concatenate([
    np.zeros(len(scores_val_clean[any_key]), dtype=int),
    np.ones(len(scores_val_adv[any_key]), dtype=int),
])

detector = api.fit_detector(scores_val_all, labels_val, config)

# Evaluate on test
scores_test_all = api.concat_scores(scores_test_clean, scores_test_adv)
any_key_t = next(iter(scores_test_all.keys()))
labels_test = np.concatenate([
    np.zeros(len(scores_test_clean[any_key_t]), dtype=int),
    np.ones(len(scores_test_adv[any_key_t]), dtype=int),
])

raw_scores_test = np.asarray(detector.score(scores_test_all), dtype=float)
metrics = api.evaluate_detection(labels_test, raw_scores_test)

thr = float(getattr(detector, "threshold", np.nan))

print("\nDetector performance (topology_mahalanobis score):")
print(f"  AUROC:        {metrics['roc_auc']:.4f}")
print(f"  AUPRC:        {metrics['pr_auc']:.4f}")
print(f"  FPR@95%TPR:   {metrics['fpr_at_tpr95']:.4f}")
print(f"  threshold:    {thr:.4f}")

# Quick plots (shared utilities)
plot_roc_from_metrics(
    metrics,
    title=f"ROC: {DATASET_NAME} ({ATTACK_TYPE}, eps={EPSILON})",
)

if np.isfinite(thr):
    plot_confusion_matrix(labels_test, y_scores=raw_scores_test, threshold=thr, labels=("clean", "adv"))

raw_clean = raw_scores_test[: len(X_test_clean_used)]
raw_adv = raw_scores_test[len(X_test_clean_used):]
plot_score_distributions_figure(
    raw_clean,
    raw_adv,
    score_name="topology_mahalanobis score",
    title="Topology score distribution (test)",
    bins=40,
    threshold=thr,
    labels=("clean", "adversarial"),
    alpha=0.6,
    density=True,
    figsize=(7, 4),
)


## Visualize Detector Steps with PH Diagrams

This section visualizes the detector pipeline step-by-step, showing:
1. Local neighborhood point cloud (in feature space)
2. Persistence diagrams (H0 and H1)
3. Topology summary features
4. Final detector score

We'll compare a clean non-flagged sample with an adversarial flagged sample.

In [ ]:
# Helper function to compute topology features with diagrams for a single point
from typing import Sequence

def compute_topology_feature_scores_with_diagrams(
    X_point: np.ndarray,
    *,
    model: nn.Module,
    Z_train: np.ndarray,
    graph_cfg,
    device: str,
    layer: str = 'penultimate',
) -> Tuple[Dict[str, float], Sequence[np.ndarray], np.ndarray]:
    """Compute topology features with diagrams for a single point (for visualization)."""
    from sklearn.neighbors import NearestNeighbors
    
    if graph_cfg.space == 'feature':
        # For images, preserve dimensions: add batch dimension without flattening
        # X_point is likely (C, H, W) or (H, W, C), we need (1, C, H, W) or (1, H, W, C)
        if X_point.ndim == 3:
            # Add batch dimension: (C, H, W) -> (1, C, H, W) or (H, W, C) -> (1, H, W, C)
            X_point_batch = X_point[None, ...]
        elif X_point.ndim == 2:
            # For 2D data (like two-moons), flatten: (D,) -> (1, D)
            X_point_batch = X_point.reshape(1, -1)
        else:
            # Already has batch dimension or is 1D
            X_point_batch = X_point if X_point.ndim > 1 else X_point.reshape(1, -1)
        Z_point = extract_features_batch(model, X_point_batch, layer=layer, device=device)[0]
    else:
        Z_point = X_point
    
    topo_cfg = TopologyConfig(
        neighborhood_k=int(getattr(graph_cfg, 'topo_k', 50)),
        maxdim=int(getattr(graph_cfg, 'topo_maxdim', 1)),
        metric=str(getattr(graph_cfg, 'topo_metric', 'euclidean')),
        thresh=getattr(graph_cfg, 'topo_thresh', None),
        min_persistence=float(getattr(graph_cfg, 'topo_min_persistence', 1e-6)),
        preprocess=str(getattr(graph_cfg, 'topo_preprocess', 'none')),
        pca_dim=int(getattr(graph_cfg, 'topo_pca_dim', 10)),
    )
    
    nbrs = NearestNeighbors(
        n_neighbors=min(topo_cfg.neighborhood_k, len(Z_train)),
        metric=topo_cfg.metric,
    ).fit(Z_train)
    
    # Get neighborhood
    _, idx = nbrs.kneighbors(Z_point.reshape(1, -1))
    neighborhood = Z_train[idx[0]]
    # Ensure the query point participates in the complex
    cloud = np.vstack([Z_point.reshape(1, -1), neighborhood])
    
    # Compute PH features with diagrams
    features, diagrams = local_persistence_features(cloud, topo_cfg, return_diagrams=True)
    
    return features, diagrams, cloud

# Import visualization functions
from src.visualization import (
    plot_persistence_diagram, 
    plot_topology_summary_features, 
    plot_local_neighborhood
)
from src.detectors import predict_graph_detector

# Select examples for visualization
# 1. Clean non-flagged sample (low detector score)
clean_scores = detector.score(scores_test_clean)
clean_idx = np.argmin(clean_scores)  # Lowest score = most clean-like
print(f"Selected clean sample: index {clean_idx}, score={clean_scores[clean_idx]:.4f}")

# 2. Adversarial flagged sample (high detector score)
adv_scores = detector.score(scores_test_adv)
adv_idx = np.argmax(adv_scores)  # Highest score among adversarial
print(f"Selected adversarial sample: index {adv_idx}, score={adv_scores[adv_idx]:.4f}")

# Get predictions for context
predictions_test, _ = predict_graph_detector(detector, scores_test_all)
any_key = next(iter(scores_test_clean.keys()))
clean_pred_idx = clean_idx
adv_pred_idx = len(scores_test_clean[any_key]) + adv_idx
print(f"Clean sample flagged: {predictions_test[clean_pred_idx]}")
print(f"Adversarial sample flagged: {predictions_test[adv_pred_idx]}")

# Compute scores with diagrams for both samples
print("\nComputing topology features with diagrams for clean sample...")
clean_features, clean_diagrams, clean_cloud = compute_topology_feature_scores_with_diagrams(
    X_test_clean_used[clean_idx], model=model, Z_train=Z_train, 
    graph_cfg=config.graph, device=DEVICE
)

print("Computing topology features with diagrams for adversarial sample...")
adv_features, adv_diagrams, adv_cloud = compute_topology_feature_scores_with_diagrams(
    X_test_adv_used[adv_idx], model=model, Z_train=Z_train, 
    graph_cfg=config.graph, device=DEVICE
)

# Get detector scores
clean_detector_score = clean_scores[clean_idx]
adv_detector_score = adv_scores[adv_idx]

print(f"\nDetector scores:")
print(f"  Clean sample: {clean_detector_score:.4f} (threshold: {detector.threshold:.4f})")
print(f"  Adversarial sample: {adv_detector_score:.4f} (threshold: {detector.threshold:.4f})")

In [ ]:
# Visualize local neighborhood point clouds
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Clean sample neighborhood
plot_local_neighborhood(
    clean_cloud,
    query_point=clean_cloud[0] if len(clean_cloud) > 0 else None,
    title=f"Clean Sample Neighborhood\n(Detector Score: {clean_detector_score:.4f})",
    ax=axes[0],
    max_dims=2
)

# Adversarial sample neighborhood
plot_local_neighborhood(
    adv_cloud,
    query_point=adv_cloud[0] if len(adv_cloud) > 0 else None,
    title=f"Adversarial Sample Neighborhood\n(Detector Score: {adv_detector_score:.4f})",
    ax=axes[1],
    max_dims=2
)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize persistence diagrams (H0 and H1)
fig, axes = plt.subplots(2, 2, figsize=(16, 16))

# Clean sample H0
if len(clean_diagrams) > 0 and clean_diagrams[0].size > 0:
    plot_persistence_diagram(
        clean_diagrams[0],
        dimension=0,
        title=f"Clean Sample - H0 Persistence Diagram\n(Score: {clean_detector_score:.4f})",
        ax=axes[0, 0],
        min_persistence=config.graph.topo_min_persistence
    )
else:
    axes[0, 0].text(0.5, 0.5, 'No H0 features', ha='center', va='center', transform=axes[0, 0].transAxes)
    axes[0, 0].set_title(f"Clean Sample - H0 Persistence Diagram\n(Score: {clean_detector_score:.4f})")

# Clean sample H1
if len(clean_diagrams) > 1 and clean_diagrams[1].size > 0:
    plot_persistence_diagram(
        clean_diagrams[1],
        dimension=1,
        title=f"Clean Sample - H1 Persistence Diagram\n(Score: {clean_detector_score:.4f})",
        ax=axes[0, 1],
        min_persistence=config.graph.topo_min_persistence
    )
else:
    axes[0, 1].text(0.5, 0.5, 'No H1 features', ha='center', va='center', transform=axes[0, 1].transAxes)
    axes[0, 1].set_title(f"Clean Sample - H1 Persistence Diagram\n(Score: {clean_detector_score:.4f})")

# Adversarial sample H0
if len(adv_diagrams) > 0 and adv_diagrams[0].size > 0:
    plot_persistence_diagram(
        adv_diagrams[0],
        dimension=0,
        title=f"Adversarial Sample - H0 Persistence Diagram\n(Score: {adv_detector_score:.4f})",
        ax=axes[1, 0],
        min_persistence=config.graph.topo_min_persistence
    )
else:
    axes[1, 0].text(0.5, 0.5, 'No H0 features', ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title(f"Adversarial Sample - H0 Persistence Diagram\n(Score: {adv_detector_score:.4f})")

# Adversarial sample H1
if len(adv_diagrams) > 1 and adv_diagrams[1].size > 0:
    plot_persistence_diagram(
        adv_diagrams[1],
        dimension=1,
        title=f"Adversarial Sample - H1 Persistence Diagram\n(Score: {adv_detector_score:.4f})",
        ax=axes[1, 1],
        min_persistence=config.graph.topo_min_persistence
    )
else:
    axes[1, 1].text(0.5, 0.5, 'No H1 features', ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title(f"Adversarial Sample - H1 Persistence Diagram\n(Score: {adv_detector_score:.4f})")

plt.tight_layout()
plt.show()

In [ ]:
# Visualize topology summary features
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Clean sample features
plot_topology_summary_features(
    clean_features,
    title=f"Clean Sample - Topology Summary Features\n(Detector Score: {clean_detector_score:.4f}, Threshold: {detector.threshold:.4f})",
    ax=axes[0]
)

# Adversarial sample features
plot_topology_summary_features(
    adv_features,
    title=f"Adversarial Sample - Topology Summary Features\n(Detector Score: {adv_detector_score:.4f}, Threshold: {detector.threshold:.4f})",
    ax=axes[1]
)

plt.tight_layout()
plt.show()

# Print feature comparison
print("\nTopology Feature Comparison:")
print("=" * 80)
for key in sorted(clean_features.keys()):
    clean_val = clean_features.get(key, 0.0)
    adv_val = adv_features.get(key, 0.0)
    diff = adv_val - clean_val
    print(f"{key:30s} Clean: {clean_val:8.4f}  Adversarial: {adv_val:8.4f}  Diff: {diff:8.4f}")
print("=" * 80)